In [28]:
import pandas as pd
import os
import numpy as np
from tqdm.notebook import tqdm
import joblib
from sklearn.neighbors import NearestNeighbors
import tmap as tm
from scipy.stats import rankdata
from matplotlib.colors import ListedColormap
from faerun import Faerun

In [2]:
elements = ["K","L",""]
branching_elements = ["B"]
enumeration = {}
enumeration[1] = [e for e in elements]

for i in range (2,16):
    if i % 4 == 0:
        elements_to_append = branching_elements
    else:
        elements_to_append = elements
    enumeration[i] = set()
    for e in elements_to_append:
        for previous_enumeration in enumeration[i-1]:
            enumeration[i].add( previous_enumeration + e )


In [3]:
len(enumeration[15])

50625

In [4]:
KL_enum = enumeration[15]

In [5]:
lib = pd.read_csv("/data/Peptides/dendrimers/T25_analogs/database/52k.mxfp", sep=" ", header=None)

In [6]:
lib.columns=["SMILES","seq", "MXFP", "unknown"]

In [7]:
lib["seq2"] = lib.seq.map(lambda x: x.replace("(", "").replace(")","").replace("8K","B").replace("4K","B").replace("2K","B"))

In [8]:
def is_in_KL_enum(seq):
    if seq[0] == "K" and "B"+seq[1:] in KL_enum:
        return True
    elif seq in KL_enum:
        return True
    else:
        return False

In [9]:
lib["in_50k"] = lib.seq2.map(is_in_KL_enum)

In [10]:
lib.query("in_50k == True")

,SMILES,seq,MXFP,unknown,seq2,in_50k
0,CC(C)CC(NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(...,(KLL)8(KLLL)4(KLLL)2KLLL,4;4;6;7;8;9;10;12;15;18;23;30;39;47;55;67;86;1...,432;260;0;53;67;16;1,KLLBLLLBLLLBLLL,True
1,CC(C)CC(NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(...,(KKL)8(KLLL)4(KLLL)2KLLL,4;4;6;7;8;9;10;12;15;18;23;29;38;47;55;67;85;1...,440;260;0;53;75;24;1,KKLBLLLBLLLBLLL,True
2,CC(C)CC(NC(=O)C(CC(C)C)NC(=O)C(CCCCNC(=O)C(CC(...,(KLL)8(KLLL)4(KLLL)2KLL,4;4;6;7;8;9;10;12;15;18;23;30;39;47;55;66;84;1...,424;255;0;52;66;16;1,KLLBLLLBLLLBLL,True
3,CC(C)CC(NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(...,(KLL)8(KLLL)4(KLL)2KLLL,4;4;6;7;8;10;10;12;15;18;23;30;39;48;57;70;90;...,416;250;0;51;65;16;1,KLLBLLLBLLBLLL,True
4,CC(C)CC(NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(...,(KLL)8(KLL)4(KLLL)2KLLL,5;4;6;7;8;10;10;12;15;19;24;31;40;50;60;74;92;...,400;240;0;49;63;16;1,KLLBLLBLLLBLLL,True
...,...,...,...,...,...,...
52520,[NH3+]CCCCC([NH3+])C(=O)NC(CCCC[NH3+])C(=O)NC(...,(KKK)4(KKKK)2K,6;6;8;9;10;12;14;16;21;26;31;37;46;58;67;67;65...,226;125;0;26;50;26;1,KKKBKKKB,True
52525,[NH3+]CCCCC([NH3+])C(=O)NC(CCCC[NH3+])C(=O)NC(...,(KKKK)4(K)2K,6;6;8;9;11;12;14;17;21;26;31;36;43;53;70;89;10...,208;115;0;24;46;24;1,KKKKBB,True
52526,[NH3+]CCCCC([NH3+])C(=O)NC(CCCC[NH3+])C(=O)NC(...,(KKKK)4(KK)2K,6;6;8;9;10;12;14;16;21;26;31;37;44;53;66;79;88...,226;125;0;26;50;26;1,KKKKBKB,True
52527,[NH3+]CCCCC([NH3+])C(=O)NC(CCCC[NH3+])C(=O)NC(...,(KKKK)4(KKK)2K,6;6;7;9;10;12;13;16;20;25;30;36;44;54;65;74;78...,244;135;0;28;54;28;1,KKKKBKKB,True


In [12]:
df = lib.query("in_50k == True").sample(frac=0.1).copy()

In [13]:
mxfps_ = df["MXFP"].tolist()
mxfps = []

for mxfp_ in mxfps_:
    mxfp = np.array(list(map(int, mxfp_.split(';'))))
    mxfps.append(mxfp)    

knn = 60

knn_search = NearestNeighbors(n_neighbors=knn, radius=1.0, algorithm='auto', leaf_size=30, metric='manhattan', p=2, metric_params=None, n_jobs=None)
knn_search.fit(np.array(mxfps))

edge_list = []

for i in tqdm(range(len(mxfps))):
    dists, idxs = knn_search.kneighbors(mxfps[i].reshape(1,-1))
    for j in range(knn):
        edge_list.append([i, idxs[0,j], dists[0,j]])

In [15]:
if not os.path.exists("/data/Peptides/dendrimers/T25_analogs/database/tm_layout_mxfp_fract.pkl"):
    # Compute the layout
    x_, y_, s, t, gp = tm.layout_from_edge_list(len(edge_list), edge_list)
    tm_layout_el = {"x": list(x_), "y": list(y_), "s" : list(s), "t" : list(t)}
    joblib.dump(tm_layout_el, "/data/Peptides/dendrimers/T25_analogs/database/tm_layout_mxfp_fract.pkl")
else:
    tm_layout_el = joblib.load("/data/Peptides/dendrimers/T25_analogs/database/tm_layout_mxfp_fract.pkl")

In [19]:
relevant_2021 = pd.read_csv("/data/Peptides/dendrimers/T25_analogs/all.smi", sep=" ", header=None)
relevant_2021.columns=["SMILES", "seq"]
relevant = relevant_2021.seq.to_list()
relevant_2018 = pd.read_csv("/data/Peptides/dendrimers/G3KL-GAanalogs/actives-from-papers", sep=" ", header=None)
relevant_2018.columns=["name", "seq"]
relevant_old = relevant_2018.seq.to_list()

In [44]:
import re
def positive(seq):
    pos = 0
    gens = re.split('1|2|3|4', seq)[::-1]
    for i, gen in enumerate(gens):
        j = 0
        if i == 0:
            j = 1
        else:
            j = 2**i
        pos += (gen.count("K") + gen.count("R"))*j
    return pos

def hydrophobic(seq):
    return seq.count("A") + seq.count("V") + seq.count("L") + seq.count("I") + seq.count("F") + seq.count("M") + seq.count("W") + seq.count("C") 

def pos_hydro_ratio(row):
    if row.hydrophobic == 0:
        return row.positive
    return round(row.positive/row.hydrophobic, 2)

def negative(seq):
    return seq.count("E") + seq.count("D")

def glu_first(seq):
    first = re.split('1|2|3|4', seq)[-2]
    return first.count("E")

def glu_core(seq):
    core = re.split('1|2|3|4', seq)[-1]
    return core.count("E")

def glu(row):
    if row.glu_core == 1:
        return 1
    elif row.glu_first == 1:
        return 2
    else:
        return 0


def generations(seq):
    seq = seq[1:]
    if seq.count("1") + seq.count("2") + seq.count("3") + seq.count("4") == 2:
        return 0
    else:
        return 1
    
def HAC(prop):
    return int(prop.split(";")[0])

def isSelected(row):
    if row.seq in relevant:
        return 2
    elif row.seq2.replace("B","4") in relevant_old:
        return 1
    else:
        return 0

mxfp_query = "5;5;7;8;9;10;11;13;17;21;27;36;44;52;64;82;100;111;104;92;61;14;0;0;0;0;0;0;0;0;0;7;5;5;4;3;4;8;10;13;16;18;26;34;41;47;62;77;86;82;72;50;12;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;16;0;0;0;0;10;1;3;8;7;12;14;15;18;24;31;37;40;34;27;14;1;0;0;0;0;0;0;0;0;0;13;0;0;8;0;1;9;4;6;9;10;12;17;21;25;31;39;44;44;43;33;10;0;0;0;0;0;0;0;0;0;21;0;0;0;0;1;6;0;0;0;0;2;5;11;11;7;11;23;29;30;46;29;2;0;0;0;0;0;0;0;0;100;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0"
b = np.array(list(map(int, mxfp_query.split(';'))))
def calc_CBD(mxfp):
    a = np.array(list(map(int, mxfp.split(';'))))
    return sum(abs(a-b))

In [45]:
df["positive"] = df.seq.map(positive)
df["negative"] = df.seq.map(negative)
df["glu_core"] = df.seq.map(glu_core)
df["glu_first"] = df.seq.map(glu_first)
df["hydrophobic"] = df.seq.map(hydrophobic)
df["pos_hydro_ratio"] = df.apply(pos_hydro_ratio, 1)
df["generations"] =  df.seq.map(generations)
df["hac"] = df.unknown.map(HAC)
df["selected"] = df.apply(isSelected,1)
df["CBD"] = df.MXFP.map(calc_CBD)

In [46]:
HAC = df.hac.tolist()
cbd = df.CBD.tolist()
cbd_r = rankdata(cbd)
seq = df.seq.tolist()
SMILES = df.SMILES.tolist()
selected = df.selected.tolist()
p_h_ratio =  df.pos_hydro_ratio.tolist()


custom_cmap = ListedColormap(['lightgray', 'magenta', "lime"], name="custom")

groups = ["0 - No", "1 - 2018", "2 - 2021"]
labels_groups, groups = Faerun.create_categories(groups)

In [47]:
faerun = Faerun(view="front", coords=False, title='MXFP_T25analogs_fract')#, clear_color="#ffffff",)
faerun.add_scatter("T25analogs",{"x": tm.VectorFloat(tm_layout_el["x"]), "y": tm.VectorFloat(tm_layout_el["y"]),\
                                "c": [HAC, cbd_r, selected, p_h_ratio],\
                          "labels": seq}, has_legend=True, \
    colormap=["rainbow", "rainbow_r", custom_cmap, "rainbow"], \
        point_scale=0.05, categorical=[False, False, True, False],\
        series_title=["HAC", "CBD (rank)", "selection", "positive/hydrophobic residues ratio"], \
            max_legend_label=[str(max(HAC)),str(max(cbd)), None, str(max(p_h_ratio))],\
                   min_legend_label=[str(min(HAC)), str(min(cbd)), None, str(min(p_h_ratio))],\
                    legend_labels=[None, None, labels_groups, None])


faerun.add_tree("T25analogs_tree", {"from": tm.VectorUint(tm_layout_el["s"]), "to": tm.VectorUint(tm_layout_el["t"])}, point_helper="T25analogs", color="aaaaaa")
faerun.plot('MXFP_T25analogs_fract')

/home/alice/Code/KL_library_enumeration/MXFP_T25analogs_fract.html